In [13]:
import pandas as pd
import re, subprocess, pyodbc 
import numpy as np # linear algebra 
from sqlalchemy import func, create_engine
from config import mongoConn, connstr  

In [59]:
engine = create_engine(connstr)
conn = engine.connect()
query = """ \
SELECT 
CONVERT(NVARCHAR(15) , IdText) as IDT
, 
REPLACE(IdText, '.','_') as IDT_
, 
CASE 
WHEN CONVERT(NVARCHAR(9),Section_Count) = '0' THEN '1a'
WHEN CONVERT(NVARCHAR(9),Section_Count) = '1' THEN '1b'
ELSE  CONVERT(NVARCHAR(9),Section_Count) 
END AS SEC 
, CONVERT(NVARCHAR(9) , QPK) as QPK
, QTEXT as QTXT 
FROM vw_MetricsCompositeKeys WHERE PK_FORM = '2021-A-HVA'
ORDER BY Section_Count ASC, SortPos ASC
""" 
df = pd.read_sql(query,con=conn)   
df.to_excel(r'C:\Users\Tim\Documents\1802\BOD1802_2021.xlsx','AllMetrics')
df=df.loc[ df.IDT_.isin(['4_1_1','6_2_1','6_2_2','6_2_3','6_2_4','6_2_4_1','6_3_1_1' ]) ]
df

,IDT,IDT_,SEC,QPK,QTXT
7,4.1.1,4_1_1,1a,22814,If Other:
19,6.2.1,6_2_1,1a,22823,Please list web browsers (client)
20,6.2.2,6_2_2,1a,22824,Please list web application servers
21,6.2.3,6_2_3,1a,22825,Please list databases
22,6.2.4,6_2_4,1a,22817,Languages/technologies/platforms is your web a...
23,6.2.4.1,6_2_4_1,1a,22822,If Other:
27,6.3.1.1,6_3_1_1,1a,22826,If Other Application Platform:


In [35]:
def hasher(t):
    t = re.sub('[^A-Za-z0-9]', '', t )     
    return t

In [58]:
aspx = '' 
for i,r in df.iterrows():  
    s = open(f'C:\\_som\\_src\\TXT.aspx', 'r').read()
    s = s.replace('{0}',str(r.QPK)).replace('{1}',str(r.IDT_)).replace('{4}',str(r.IDT_)+ ' : ' +str(r.QPK)+ '\n\t' +str(r.QTXT)).replace('{5}','')
    s = s.replace('{2}',str(160))
    df.loc[i, 'ASPX'] = s
    aspx=aspx+s+ '\n\n'
    print(df.loc[i, 'ASPX'] + '\n\n') 
  
with open(r'C:\\_som\\_cache.txt', 'w') as file:
  file.write(aspx)

<!-- 
    4_1_1 : 22814
	If Other:
--->
<tr id="r-m-4_1_1" class="" style="display:<%= _ShowRow_4_1_1  %>">
    <td class="LabelNoWrap" style="width: 4%; text-align: left">
        <uc1:CBlabel CSSClass="CustomControlLabelNoVertAlign" ID="CBlabel_4_1_1_i" runat="server" LabelType="QuestionNumber"
            LinkEditID="CBtext_4_1_1" />
    </td>
    <td class="LabelColumn" colspan="3" style="width: 73%; text-align: left">
        <uc1:CBlabel CSSClass="CustomControlLabelNoVertAlign" ID="CBlabel_4_1_1_ii" runat="server" LabelType="Question"
            LinkEditID="CBtext_4_1_1" /> 
        <asp:Label ID="Label_4_1_1_i" runat="server" Text="" CssClass="CustomControlLabelNoVertAlign"></asp:Label>
    </td>
    <td class="ControlColumn" style="width: 17%; text-align: right">
        <uc2:CBtext2 ID="CBtext_4_1_1" runat="server" CSSClass="CustomControlValue" EditState="ReadMode"
            IsReadOnly="false" NullDisplay="Blank" PK_Question="22814" UseSpellCheck="false" MaxLength="1500"
  

In [194]:
sr=''
for i,r in df_mas.iterrows():
    sr = sr + showrow.replace('{0}', r.ID.replace(".","_"))
print(sr)

    Protected _ShowRow_12 As String
    Public Property ShowRow_12() As String
        Get
            Return _ShowRow_12
        End Get
        Set(ByVal value As String)
            _ShowRow_12 = value
        End Set
    End Property
    
    Protected _ShowRow_12_1 As String
    Public Property ShowRow_12_1() As String
        Get
            Return _ShowRow_12_1
        End Get
        Set(ByVal value As String)
            _ShowRow_12_1 = value
        End Set
    End Property
    
    Protected _ShowRow_12_2 As String
    Public Property ShowRow_12_2() As String
        Get
            Return _ShowRow_12_2
        End Get
        Set(ByVal value As String)
            _ShowRow_12_2 = value
        End Set
    End Property
    
    Protected _ShowRow_12_3 As String
    Public Property ShowRow_12_3() As String
        Get
            Return _ShowRow_12_3
        End Get
        Set(ByVal value As String)
            _ShowRow_12_3 = value
        End Set
    End Property
    
    

In [135]:
df_picks = pd.DataFrame(); 
df = pd.read_csv(fr'c:\Users\Tim\Documents\1802\s1.csv')   
df = df.fillna('~')
df=df.loc[ df.TYPE != '~' ] 
df=df.loc[ df.METRIC.str.contains('PL-')  ] 
pickseed = 3980
sorder=0
METRIC=df.iloc[0]['METRIC']
for i,r in df.iterrows():

    if METRIC != r.METRIC:
        sorder=0
        METRIC=r.METRIC 
    pickseed = pickseed + 1 
    sorder = sorder + 1 
    TYPE = re.sub('\s\s', '', r.TYPE)  
    TYPE= re.sub('\(Free Text\)', '', TYPE) 
    # TYPE= re.sub(', ', '', TYPE) 
    TYPE= re.sub('· ', '', TYPE) 
    CodeValue=r.TYPE.upper() 
    CodeValue = re.sub('[^A-Z0-9]', '', CodeValue)   
    if len(CodeValue) > 4 :
        CodeValue = re.sub('[AEIOUY]', '', CodeValue)
        CodeValue=CodeValue[:4]+CodeValue[-4:]
        if re.search('.*Other.*', TYPE):
            CodeValue="OTH"  
    
    TYPE = TYPE.replace(", "," ")
    df.loc[i, 'PK_Picklist'] = f"({pickseed},@PKT, N'{CodeValue}','{TYPE}',{sorder}, GETDATE(), 1),"
    print(df.loc[i, 'PK_Picklist']) 
    
df.head(59)

(3981,@PKT, N'5RS5RS','< 5 years',1, GETDATE(), 1),
(3982,@PKT, N'610R10RS','> 6 – 10 years',2, GETDATE(), 1),
(3983,@PKT, N'101515RS','> 10 – 15 years',3, GETDATE(), 1),
(3984,@PKT, N'152020RS','> 15 – 20 years',4, GETDATE(), 1),
(3985,@PKT, N'203030RS','> 20 – 30 years',5, GETDATE(), 1),
(3986,@PKT, N'304040RS','> 30 – 40 years',6, GETDATE(), 1),
(3987,@PKT, N'40RS40RS','> 40 years',7, GETDATE(), 1),
(3988,@PKT, N'LGCCMMPS','Legacy codebase (COBOL MUMPS)',1, GETDATE(), 1),
(3989,@PKT, N'LCKFNTTN','Lack of or poor documentation',2, GETDATE(), 1),
(3990,@PKT, N'LGCNRCTR','Legacy infrastructure',3, GETDATE(), 1),
(3991,@PKT, N'DHCNRCTR','Ad-hoc infrastructure ',4, GETDATE(), 1),
(3992,@PKT, N'HRDWPPRT','Hardware or software with no vendor/manufacturer support',5, GETDATE(), 1),
(3993,@PKT, N'TDTDVNTR','Outdated or inaccurate inventory',6, GETDATE(), 1),
(3994,@PKT, N'TCHNLGTP','Technology Type ',7, GETDATE(), 1),
(3995,@PKT, N'SSTMRNZD','System Last Modernized',8, GETDATE(), 1),
(3996,@

,ID,METRIC,QTEXT,TYPE,PK_Picklist
4,~,PL-APPDPLY,~,· < 5 years,"(3981,@PKT, N'5RS5RS','< 5 years',1, GETDATE()..."
5,~,PL-APPDPLY,~,· > 6 – 10 years,"(3982,@PKT, N'610R10RS','> 6 – 10 years',2, GE..."
6,~,PL-APPDPLY,~,· > 10 – 15 years,"(3983,@PKT, N'101515RS','> 10 – 15 years',3, G..."
7,~,PL-APPDPLY,~,· > 15 – 20 years,"(3984,@PKT, N'152020RS','> 15 – 20 years',4, G..."
8,~,PL-APPDPLY,~,· > 20 – 30 years,"(3985,@PKT, N'203030RS','> 20 – 30 years',5, G..."
9,~,PL-APPDPLY,~,· > 30 – 40 years,"(3986,@PKT, N'304040RS','> 30 – 40 years',6, G..."
10,~,PL-APPDPLY,~,· > 40 years,"(3987,@PKT, N'40RS40RS','> 40 years',7, GETDAT..."
26,~,PL-LGYCHR,~,"Legacy codebase (COBOL, MUMPS)","(3988,@PKT, N'LGCCMMPS','Legacy codebase (COBO..."
27,~,PL-LGYCHR,~,Lack of or poor documentation,"(3989,@PKT, N'LCKFNTTN','Lack of or poor docum..."
28,~,PL-LGYCHR,~,Legacy infrastructure,"(3990,@PKT, N'LGCNRCTR','Legacy infrastructure..."


In [ ]:
 df = pd.read_csv(fr'c:\Users\Tim\Documents\1802\BOD_2021_Metrics.csv')  

In [189]:
query = """\
SELECT PK_PickList, PK_PickListType, DisplayValue, CodeValue, SortPos, Description, UsageField FROM vwPickLists
WHERE PK_PickListType IN(160 )
"""
dfpicks = pd.read_sql(query,con=conn) 
UsageField=dfpicks.iloc[0]['UsageField']
s=f"\n"
s = s + f"Enum PICK_{UsageField} \t'{dfpicks.iloc[0]['Description']}-{dfpicks.iloc[0]['PK_PickListType']}\n"
for i,r in dfpicks.iterrows():
    if UsageField != dfpicks.iloc[i]['UsageField']:
        UsageField=dfpicks.iloc[i]['UsageField']
        s = s + f"End Enum\n"
        s = s + f"Enum PICK_{UsageField} \t'{dfpicks.iloc[i]['Description']}-{dfpicks.iloc[i]['PK_PickListType']}\n"
    s = s + f"\t{r.CodeValue} = {str(r.PK_PickList)} \t'{str(r.DisplayValue)}\n" 
s=s+"End Enum\n"
print(s)


Enum PICK_HVAAGE 	'HVA Age-160
	ONE = 3471 	'< 1 Year Old
	THREE = 3472 	'1 - 3 Years Old
	FIVE = 3473 	'>3 - 5 Years old
	FIVEPLUS = 3474 	'>5 Years
	UNK = 3475 	'Unknown/Never
End Enum



In [147]:
showrow = """\
    Protected _ShowRow_{0} As String
    Public Property ShowRow_{0}() As String
        Get
            Return _ShowRow_{0}
        End Get
        Set(ByVal value As String)
            _ShowRow_{0} = value
        End Set
    End Property
    
"""

In [ ]:
import nltk
stop_words = set(stopwords.words('english'))  

In [4]:
df = pd.read_csv(fr'c:\Users\Tim\Documents\1802\HVA_20_21_ID_Mapper.csv')  
#df.to_sql('HVA_IDTEXT_Map', engine,  if_exists='replace', index=False )

df

,IDT_2020,IDT_2021,QTEXT
0,1,1,Whast is the purpose of the HVA?
1,2,2,What services does the HVA system provide?
2,2.1,2.1,To whom does the HVA provide services to?
3,3,6,System Technology Type
4,3.1,6.5,"If Other is selected, please define"
5,3.2,6.6,"If System Technology tye is a cloud, then indi..."
6,3.3,6.7,"If System Technology type is ICS/SCADA, then i..."
7,3.4,6.7.1,"If System Technology type is ICS/SCADA, then d..."
8,4,7,HVA system managent type
9,4.1,7.1,"If Other is selected, please define"


In [60]:
conn.close()